![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Analyse de Tesla (TSLA)
Ce notebook utilise les données de Tesla (TSLA) pour effectuer une analyse technique basée sur les indicateurs techniques, notamment les EMA (Exponential Moving Average), RSI (Relative Strength Index) et ATR (Average True Range).

Nous allons :
1. Extraire les données historiques pour TSLA.
2. Calculer les indicateurs techniques (EMA, RSI, ATR).
3. Analyser les croisements des EMA pour générer des signaux d'achat et de vente.
4. Optimiser les paramètres des EMA à l'aide d'une simulation.

In [21]:
## Importations nécessaires

# Imports nécessaires pour la recherche
from AlgorithmImports import *
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Initialisation de l'environnement de recherche
Définissons l'environnement de recherche QuantConnect, les périodes, et chargeons les données pour Tesla (TSLA).

In [1]:
# Initialisation de l'environnement de recherche
qb = QuantBook()
start_date = datetime(2020, 1, 1)
end_date = datetime(2023, 1, 1)

# Chargement des données historiques pour Tesla (TSLA)
symbol = qb.AddEquity("TSLA", Resolution.Daily).Symbol
history = qb.History(symbol, start_date, end_date, Resolution.Daily)

# Vérification si des données ont été récupérées
if history.empty:
    print("Aucune donnée n'a été récupérée. Vérifiez les paramètres ou la connexion à QuantConnect.")
else:
    print(f"Données chargées pour {symbol}: {len(history)} lignes.")

# Préparer les données pour une meilleure manipulation
try:
    # Sélection des colonnes d'intérêt
    history = history[['open', 'high', 'low', 'close', 'volume']]
    history = history.loc[symbol]  # Extraire les données pour l'actif TSLA uniquement
    history = history.dropna()  # Supprimer les lignes contenant des valeurs NaN

    # Vérification des colonnes disponibles après préparation
    print("\nColonnes disponibles dans le DataFrame :")
    print(history.columns)

    # Aperçu des premières lignes des données pour validation
    print("\nAperçu des données historiques préparées :")
    print(history.head())

except Exception as e:
    print(f"Erreur lors de la préparation des données : {e}")

## Calcul des Indicateurs Techniques : EMA, RSI et ATR
Nous allons calculer les indicateurs suivants :
- **EMA (Exponential Moving Average)** : Moyennes mobiles exponentielles (rapide et lente).
- **RSI (Relative Strength Index)** : Indicateur de force relative.
- **ATR (Average True Range)** : Indicateur de volatilité.

In [2]:
# Calcul manuel des EMA
history['FastEMA'] = history['close'].ewm(span=10, adjust=False).mean()  # EMA rapide (10 jours)
history['SlowEMA'] = history['close'].ewm(span=50, adjust=False).mean()  # EMA lente (50 jours)

In [ ]:
# Calcul manuel du RSI
def calculate_rsi(series, period=14):
    delta = series.diff()
    gain = delta.where(delta > 0, 0).rolling(window=period).mean()
    loss = -delta.where(delta < 0, 0).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

history['RSI'] = calculate_rsi(history['close'], 14)

In [4]:
# Calcul manuel de l'ATR
def calculate_atr(high, low, close, period=14):
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr

history['ATR'] = calculate_atr(history['high'], history['low'], history['close'], period=14)

## Aperçu des données calculées
Vérifions que les indicateurs ont été correctement ajoutés aux données historiques.

In [5]:
# Vérification des nouvelles colonnes
print("Aperçu des données avec indicateurs calculés :")
print(history[['close', 'FastEMA', 'SlowEMA', 'RSI', 'ATR']].tail())

## Analyse des Croisements EMA
Détectons des signaux d'achat (FastEMA > SlowEMA) et de vente (FastEMA < SlowEMA) en fonction des croisements des EMA rapides et lentes.

In [29]:
# Générer les signaux d'achat et de vente
history['Signal'] = 0
history.loc[history['FastEMA'] > history['SlowEMA'], 'Signal'] = 1  # Signal d'achat
history.loc[history['FastEMA'] < history['SlowEMA'], 'Signal'] = -1 # Signal de vente

## Visualisation des Données et Signaux
Créons une visualisation montrant le prix de clôture, les EMA (rapide et lente), ainsi que les signaux générés.

In [30]:
plt.figure(figsize=(14, 7))
plt.plot(history.index, history['close'], label="Prix de clôture", alpha=0.5)
plt.plot(history.index, history['FastEMA'], label="EMA rapide (10 jours)", linestyle="--", color='green')
plt.plot(history.index, history['SlowEMA'], label="EMA lente (50 jours)", linestyle="--", color='red')
plt.title("Tesla (TSLA) - Analyse des EMAs et des signaux")
plt.legend()
plt.show()

## Optimisation des Périodes EMA
Testons différentes combinaisons d'EMA rapide et lente pour maximiser le profit.

In [6]:
def optimize_ema(history, fast_range, slow_range):
    best_combination = None
    best_profit = -float('inf')
    results = []

    for fast in fast_range:
        for slow in slow_range:
            if fast >= slow:
                continue

            # Calcul des EMA ajustées
            history['FastEMA'] = history['close'].ewm(span=fast, adjust=False).mean()
            history['SlowEMA'] = history['close'].ewm(span=slow, adjust=False).mean()

            # Génération des signaux
            history['Signal'] = 0
            history.loc[history['FastEMA'] > history['SlowEMA'], 'Signal'] = 1
            history.loc[history['FastEMA'] < history['SlowEMA'], 'Signal'] = -1

            # Calcul des performances
            history['DailyPnL'] = history['Signal'].shift(1) * history['close'].pct_change()
            cumulative_return = (1 + history['DailyPnL']).cumprod().iloc[-1]

            results.append((fast, slow, cumulative_return))

            if cumulative_return > best_profit:
                best_profit = cumulative_return
                best_combination = (fast, slow)

    return best_combination, best_profit, pd.DataFrame(results, columns=['FastEMA', 'SlowEMA', 'CumulativeReturn'])

# Tester des plages de valeurs pour les EMA
fast_range = range(5, 30, 5)
slow_range = range(30, 100, 10)
best_comb, best_profit, results_df = optimize_ema(history, fast_range, slow_range)

print(f"Meilleure combinaison EMA rapide/lente : {best_comb} avec un profit cumulé de {best_profit:.2%}")

In [12]:
# Pivot et heatmap
results_pivot = results_df.pivot(index="FastEMA", columns="SlowEMA", values="CumulativeReturn")
plt.figure(figsize=(12, 7))
sns.heatmap(results_pivot, annot=True, fmt=".1%", cmap="coolwarm", cbar_kws={'label': 'Cumulative Return'})
plt.title("Optimisation des périodes EMA")
plt.xlabel("Slow EMA Period (jours)")
plt.ylabel("Fast EMA Period (jours)")
plt.show()

## Analyse des Résultats : Tesla (TSLA)
Ce script analyse :
1. Les valeurs maximales, minimales et moyennes des prix.
2. Les signaux d'achat et de vente générés par les indicateurs (EMA).
3. Une visualisation des signaux sur les graphiques des prix.
4. Statistiques financières liées aux signaux (taux de réussite, gain moyen, perte moyenne, performance cumulée).

In [8]:
# Analyse des Résultats : Tesla (TSLA)
import numpy as np

# Vérification des données nécessaires
if 'Signal' not in history or 'FastEMA' not in history or 'SlowEMA' not in history:
    print("Erreur : Les signaux ou indicateurs EMA ne sont pas présents dans les données.")
else:
    # Résumé des prix
    close_price = history['close']
    print("Résumé des Prix :")
    print(f"- Prix maximal : {close_price.max():.2f} USD")
    print(f"- Prix minimal : {close_price.min():.2f} USD")
    print(f"- Prix moyen : {close_price.mean():.2f} USD")

    # Analyse des Signaux
    buy_signals = len(history[history['Signal'] == 1])  # Nombre de signaux d'achat
    sell_signals = len(history[history['Signal'] == -1])  # Nombre de signaux de vente
    print("\nRésumé des Signaux :")
    print(f"- Signaux d'achat : {buy_signals}")
    print(f"- Signaux de vente : {sell_signals}")
    print(f"- Total des signaux : {buy_signals + sell_signals}")

    # Performance des Signaux
    history['DailyPnL'] = history['Signal'].shift(1) * history['close'].pct_change()  # PnL journalier
    history['CumulativeReturn'] = (1 + history['DailyPnL']).cumprod()  # Retour cumulé
    total_return = history['CumulativeReturn'].iloc[-1] - 1  # Gain net
    avg_gain = history[history['DailyPnL'] > 0]['DailyPnL'].mean()  # Gain moyen
    avg_loss = history[history['DailyPnL'] < 0]['DailyPnL'].mean()  # Perte moyenne
    win_rate = len(history[history['DailyPnL'] > 0]) / len(history[history['DailyPnL'].notnull()])  # Taux de gain

    print("\nPerformance des Signaux :")
    print(f"- Retour cumulé : {total_return * 100:.2f} %")
    print(f"- Gain moyen : {avg_gain * 100:.2f} %")
    print(f"- Perte moyenne : {avg_loss * 100:.2f} %")
    print(f"- Taux de gain : {win_rate * 100:.2f} %")

    # Graphique des résultats
    import matplotlib.pyplot as plt

    # Visualisation des Signaux et du Retour Cumulé
    plt.figure(figsize=(14, 7))

    # Graphique 1 : Prix avec les Signaux
    plt.subplot(2, 1, 1)
    plt.plot(history.index, history['close'], label='Prix de clôture', alpha=0.6, color='blue')
    plt.scatter(history.index[history['Signal'] == 1], history['close'][history['Signal'] == 1],
                label='Signal Achat', marker='^', color='green')
    plt.scatter(history.index[history['Signal'] == -1], history['close'][history['Signal'] == -1],
                label='Signal Vente', marker='v', color='red')
    plt.title("Prix et Signaux d'Achat/Vente (EMA)")
    plt.legend()

    # Graphique 2 : Retour Cumulé
    plt.subplot(2, 1, 2)
    plt.plot(history.index, history['CumulativeReturn'], label='Retour Cumulé', color='purple')
    plt.title("Retour Cumulé Basé sur les Signaux")
    plt.legend()

    plt.tight_layout()
    plt.show()